# Choice Set space for Neural Data

 The idea is to create a choice set that: samples the same SV difference (+ - some variation) with different trials  

In [11]:
#Import the relevant libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

# 1. Input: patient's alpha and beta parameters

In [12]:
#Define files & data pathway
patient_code = 'patient1_calibration'
patient_split_dir = '/Volumes/UCDN/Santiago_Guardo/data/split'
utility_dir = '/Volumes/UCDN/Santiago_Guardo/data/utility'
utility_analysis = '/Volumes/UCDN/Santiago_Guardo/data/utility/split_CRDM_analysis.csv'
parameters = pd.read_csv(utility_analysis)
parameters

,Unnamed: 0,subject,task,domain,response_rate,percent_lottery,percent_risk,percent_ambiguity,conf_1,conf_2,conf_3,conf_4,negLL,gamma,beta,alpha,at_bound,LL,LL0,AIC,BIC,R2,correct,prob_span,fig_fn
0,0,patient1_calibration,CRDM,gain,0.839286,0.361702,0.333333,0.411765,1,5,23,18,31.032613,0.890861,-0.391804,0.128615,0,-31.032613,-32.577917,68.065227,69.765522,0.047434,0.617021,0.222320,patient1_calibration/crdm/patient1_calibration...
1,1,patient1_calibration,CRDM,loss,0.750000,0.547619,0.461538,0.687500,0,11,15,16,28.382892,0.135825,4.167000,0.415048,1,-28.382892,-29.112182,62.765783,64.241122,0.025051,0.523810,0.290221,patient1_calibration/crdm/patient1_calibration...


In [10]:

#Define the parameter values for the given participant. 
filtered_parameters = parameters[parameters['subject'] == patient_code]

# Create alpha_pos and alpha_neg variables
alpha_pos = filtered_parameters.loc[filtered_parameters['domain'] == 'gain', 'alpha'].values[0]
alpha_neg = filtered_parameters.loc[filtered_parameters['domain'] == 'loss', 'alpha'].values[0]

beta_pos = filtered_parameters.loc[filtered_parameters['domain'] == 'gain', 'beta'].values[0]
beta_neg = filtered_parameters.loc[filtered_parameters['domain'] == 'loss', 'beta'].values[0]

print("alpha_pos:", alpha_pos)
print("alpha_neg:", alpha_neg)

print("beta_pos:", beta_pos)
print("beta_pos:", beta_neg)



alpha_pos: 0.1286154715090123
alpha_neg: 0.415047662498603
beta_pos: -0.3918035142034978
beta_pos: 4.167
